In [1]:
%matplotlib inline
!pip install scikit-learn
!pip install seaborn
!pip install geopandas matplotlib

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from glob import glob
import geopandas as gpd
import re

from scipy.spatial import distance_matrix
from sklearn.preprocessing import MinMaxScaler

# Para no mostrar 'Future Warnings' producto de que algunos paquetes aún
# trabajan con versiones no actualizadas de otros paquetes
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
DATOS_generales = pd.read_csv('./datos_proyecto_productos_filtrados.csv')
DATOS_generales['Item Name'] = DATOS_generales['Item Name'].astype(str)
DATOS_generales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6373 entries, 0 to 6372
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Sale Date        6373 non-null   object 
 1   Item Name        6373 non-null   object 
 2   Buyer            5559 non-null   object 
 3   Quantity         6373 non-null   int64  
 4   Price            6373 non-null   float64
 5   Discount Amount  6373 non-null   float64
 6   Ship City        6373 non-null   object 
 7   Ship Country     6373 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 398.4+ KB


In [3]:
def extraer_palabras_clave(texto):
    palabras_clave = re.findall(r'\b\w{4,}\s+\w{4,}\b', texto)
    return palabras_clave

In [4]:
DATOS_generales['Palabras_clave'] = DATOS_generales['Item Name'].apply(extraer_palabras_clave)

In [5]:
DATOS_generales['Palabras_clave']

0                      [Niall Horan, Holographic Sticker]
1                      [Niall Horan, Holographic Sticker]
2                      [Niall Horan, Holographic Sticker]
3                      [Niall Horan, Holographic Sticker]
4                      [Niall Horan, Holographic Sticker]
                              ...                        
6368                                        [Louis Tommo]
6369                                    [louis tomlinson]
6370    [Sunset Curve, Holographic Sticker, Sunset Curve]
6371    [Sunset Curve, Holographic Sticker, Sunset Curve]
6372    [Louis tomlinson, elastics bracelets, away fro...
Name: Palabras_clave, Length: 6373, dtype: object

In [6]:
palabras_filtro = ['camiseta','collar','holographic','tela', 'bundle','pack','tote','colgante','custom','cumpleaños', 'merch'
                  ,'with', 'from', 'best', 'pegatina', 'haces', 'pulsera', 'order', 'acrylic', 'logo', 'photo', 'eres',
                   'tattoo', 'estuche', 'banderas', 'transparent', 'elastic', 'color', 'random', 'line','sunset', 'rubber',
                  'this', 'weather', 'personalizado', 'bolsa', 'chulapas', 'keychain', 'necklace', 'lisence', 'sticker',
                   'mascarilla', 'equipaje','hand', 'llavero', 'bracelet','LEER', 'toiletry', 'ring', 'costume', 'young',
                  'bisutería', 'DESCRIPTION', 'personalizada', 'terciopelo', 'restocking', 'botella', 'para', 'presale',
                  'effect', 'simple', 'Song', 'riñonera', 'ganadores', 'laptop', 'days', 'colores', 'chromatica', 'shirt',
                  'holder', 'bottle', 'Personalized', 'face', 'acero', 'jewelry', 'colored', 'mess']
set_palabras_clave = set()
for i in DATOS_generales['Palabras_clave']:
    for j in i:
        palabra1, palabra2 = j.split()
        if palabra1.lower() not in palabras_filtro and palabra2.lower() not in palabras_filtro:
            set_palabras_clave.add(j)
print(len(set_palabras_clave),set_palabras_clave)

104 {'tour 2023', 'efecto desteñido', 'Louis smiley', '1989 taylor', 'Eddie Munson', 'gets ugly', 'niall horan', 'SALE Harry', 'lovers 2022', 'stainless steel', 'Louis afhf', 'Larry stylinson', 'Styles Fine', 'Harry styles', 'Niall Horan', 'lovers Niall', 'night talking', 'shawn mendes', 'Treat People', 'Styles jewelery', 'Madrid España', 'Niall horan', 'bracelets inspired', 'desteñido verde', 'Spain magazine', 'Taylors Version', 'Styles Barcelona', 'rosalia motomami', 'Justin bieber', 'Photocards holders', 'signed autographed', 'eras tour', 'Louis tomlison', 'years anniversary', 'Tour 2023', 'louis tommo', 'would like', 'Madrid Spain', 'elastics bracelets', 'girl almithy', 'Louis tomlinson', 'Maneskin cloth', 'summer without', 'Zayn Malik', 'Royal Albert', 'LEER DESCRIPCIÓN', 'Harry Styles', 'Barcelona España', 'Listening Album', 'summer harry', 'Mystery Harry', 'Olivia Rodrigo', 'Taylor swift', 'Erfreuliche benutzerdefinierte', 'Heaven stainless', 'love louis', 'harry styles', 'Heave

In [7]:
etiquetas_clave = ['louis', 'taylor', 'switf', 'maneskin', 'harry', 'Harrychella ', 'niall', 'stray', 'kids', 'olivia',
                   'rodrigo', 'madrid', 'barcelona', 'zayn', 'malik', 'justin', 'bieber', 'shawn', 'mendes', 'tour',
                  'liam', 'payne', 'eurovision','tour Barcelona', 'tour Madrid', 'Tour 2022', 'tour 2023', '1d', 'one direction'
                  ,'one', 'direction']
etiquetas_clave = [cadena.lower() for cadena in etiquetas_clave]

In [8]:
df_filtrado = DATOS_generales[DATOS_generales['Palabras_clave'].apply(lambda lista: any(any(palabra.lower() in etiquetas_clave for palabra in cadena.split()) for cadena in lista))]

In [9]:
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4689 entries, 0 to 6372
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Sale Date        4689 non-null   object 
 1   Item Name        4689 non-null   object 
 2   Buyer            4101 non-null   object 
 3   Quantity         4689 non-null   int64  
 4   Price            4689 non-null   float64
 5   Discount Amount  4689 non-null   float64
 6   Ship City        4689 non-null   object 
 7   Ship Country     4689 non-null   object 
 8   Palabras_clave   4689 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 366.3+ KB


In [10]:
porc_representado = (4689/6372)*100
print("El porcentaje representado por el filtro de palabras clave es de: ", porc_representado)

El porcentaje representado por el filtro de palabras clave es de:  73.58757062146893


In [11]:
def aplicar_filtro(lista):
    # Si la lista tiene una sola cadena, devolver esa cadena
    if len(lista) == 1:
        return lista[0]
    else:
        # Aplicar el filtro y devolver la cadena seleccionada
        for cadena in lista:
            if cadena.split(" ")[0] in etiquetas_clave or cadena.split(" ")[1] in etiquetas_clave or cadena in etiquetas_clave:  # Reemplaza 'filtro' con tu filtro específico
                return cadena

# Aplicar la función personalizada a cada fila de la columna 'Columna'
df_filtrado['Palabras_clave'] = df_filtrado['Palabras_clave'].apply(aplicar_filtro)

C:\Users\Carlosim\AppData\Local\Temp\ipykernel_6304\3855776682.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['Palabras_clave'] = df_filtrado['Palabras_clave'].apply(aplicar_filtro)


In [12]:
df_filtrado['Palabras_clave'].unique()

array([None, 'liam merch', 'direction merch', 'harry styles',
       'zayn merch', 'Zayn Malik', 'PACK zayn', 'direction Holographic',
       'ganadores eurovision', 'taylor swift', 'eurovision 2021',
       'FREE niall', 'louis tomlinson', 'maneskin acrylic',
       'olivia rodrigo', 'taylor switf', 'niall horan', 'merch niall',
       'tela Louis', 'summer harry', 'Pulsera Harry', 'shawn mendes',
       'Collar harry', 'Justin bieber', 'tela harry', 'tour elastics',
       'louis tommo', 'future louis', 'Maneskin cloth', 'love louis',
       'Louis Tommo', 'eras tour', 'tour Madrid', 'tour Barcelona',
       'tour 2023', '1989 taylor', 'stray kids'], dtype=object)

In [13]:
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4689 entries, 0 to 6372
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Sale Date        4689 non-null   object 
 1   Item Name        4689 non-null   object 
 2   Buyer            4101 non-null   object 
 3   Quantity         4689 non-null   int64  
 4   Price            4689 non-null   float64
 5   Discount Amount  4689 non-null   float64
 6   Ship City        4689 non-null   object 
 7   Ship Country     4689 non-null   object 
 8   Palabras_clave   3168 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 366.3+ KB


In [25]:
diccionario_etiquetas_mejor_mes = dict()

In [97]:
#Prueba que devuelve el mes con mayores ventas
etiqueta_clave = '1989 taylor'
partes_etiqueta = etiqueta_clave.split()

# Convertir las etiquetas de búsqueda a minúsculas
etiqueta_clave = etiqueta_clave.lower()

# Filtrar el DataFrame para las filas que contienen alguna de las partes de la etiqueta clave en minúsculas
df_resultado = df_filtrado[(df_filtrado['Palabras_clave'].str.lower().str.contains(partes_etiqueta[0].lower()))
                           | (df_filtrado['Palabras_clave'].str.lower().str.contains(partes_etiqueta[0].lower()))]

# Convertir la columna 'Fecha' al formato datetime si aún no lo está
df_resultado['Sale Date'] = pd.to_datetime(df_resultado['Sale Date'])

# Agrupar por mes y año en la columna 'Fecha' y contar el número de filas en cada grupo
conteo_por_mes_y_año = df_resultado.groupby([df_resultado['Sale Date'].dt.year, df_resultado['Sale Date'].dt.month]).size()

# Seleccionar el mes con más filas
mes_año_con_mas_filas = conteo_por_mes_y_año.idxmax()

total_filas_mes_mas_fila = conteo_por_mes_y_año.max()
diccionario_etiquetas_mejor_mes[etiqueta_clave] = (total_filas_mes_mas_fila, mes_año_con_mas_filas)
# Imprimir el resultado
print("Mes con más filas que cumplen el valor concreto de la columna 'Palabras_clave':", mes_año_con_mas_filas)
print("Número total de filas para ese mes y año:", total_filas_mes_mas_fila)

Mes con más filas que cumplen el valor concreto de la columna 'Palabras_clave': (2023, 8)
Número total de filas para ese mes y año: 2


C:\Users\Carlosim\AppData\Local\Temp\ipykernel_6304\133281482.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resultado['Sale Date'] = pd.to_datetime(df_resultado['Sale Date'])


In [98]:
diccionario_etiquetas_mejor_mes

{'harry styles': (314, (2023, 7)),
 'zayn malik': (36, (2021, 4)),
 'louis tomlinson': (27, (2022, 8)),
 'taylor switf': (23, (2021, 9)),
 'shawn mendes': (64, (2022, 7)),
 'niall horan': (15, (2022, 12)),
 'justin bieber': (1, (2022, 6)),
 'eras tour': (66, (2023, 8)),
 '1989 taylor': (2, (2023, 8)),
 'tour madrid': (5, (2023, 7)),
 'tour barcelona': (8, (2023, 8)),
 'stray kids': (1, (2023, 9)),
 'tour 2023': (7, (2023, 8))}

In [60]:
diccionario_ordenado = dict(sorted(diccionario_etiquetas_mejor_mes.items(), key=lambda item: item[1], reverse=True))
diccionario_ordenado

{'harry styles': (314, (2023, 7)),
 'eras tour': (66, (2023, 8)),
 'shawn mendes': (64, (2022, 7)),
 'zayn malik': (36, (2021, 4)),
 'louis tomlinson': (27, (2022, 8)),
 'taylor switf': (23, (2021, 9)),
 '1989 taylor': (23, (2021, 9)),
 'niall horan': (15, (2022, 12)),
 'tour barcelona': (8, (2023, 8)),
 'tour 2023': (7, (2023, 8)),
 'tour madrid': (5, (2023, 7)),
 'stray kids': (1, (2023, 9)),
 'justin bieber': (1, (2022, 6))}

In [79]:
diccionario_etiquetas_mejores_meses = dict()

In [109]:
#Prueba que devuelve los 3 meses con más ventas
etiqueta_clave = 'justin bieber'
partes_etiqueta = etiqueta_clave.split()

# Convertir las etiquetas de búsqueda a minúsculas
etiqueta_clave = etiqueta_clave.lower()

# Filtrar el DataFrame para las filas que contienen alguna de las partes de la etiqueta clave en minúsculas
df_resultado = df_filtrado[(df_filtrado['Palabras_clave'].str.lower().str.contains(partes_etiqueta[0].lower()))
                           | (df_filtrado['Palabras_clave'].str.lower().str.contains(partes_etiqueta[1].lower()))]

# Convertir la columna 'Fecha' al formato datetime si aún no lo está
df_resultado['Sale Date'] = pd.to_datetime(df_resultado['Sale Date'])

# Agrupar por mes y año en la columna 'Fecha' y contar el número de filas en cada grupo
conteo_por_mes_y_año = df_resultado.groupby([df_resultado['Sale Date'].dt.year, df_resultado['Sale Date'].dt.month]).size()

# Seleccionar el mes con más filas
meses_año_con_mas_filas = conteo_por_mes_y_año.nlargest(3).index.tolist()
total_filas_mes_mas_fila = conteo_por_mes_y_año.nlargest(3).tolist()
resumen = list(zip(meses_año_con_mas_filas,total_filas_mes_mas_fila))

diccionario_etiquetas_mejores_meses[etiqueta_clave] = resumen
# Imprimir el resultado
print("Los meses con más filas que cumplen el valor concreto de la columna 'Palabras_clave':", meses_año_con_mas_filas)
print("Número total de filas para ese mes y año:", total_filas_mes_mas_fila)

Los meses con más filas que cumplen el valor concreto de la columna 'Palabras_clave': [(2022, 6)]
Número total de filas para ese mes y año: [1]


C:\Users\Carlosim\AppData\Local\Temp\ipykernel_6304\921496624.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resultado['Sale Date'] = pd.to_datetime(df_resultado['Sale Date'])


In [110]:
diccionario_etiquetas_mejores_meses

{'harry styles': [((2023, 7), 314), ((2023, 8), 239), ((2022, 8), 162)],
 'eras': [((2023, 8), 66), ((2023, 9), 47), ((2023, 7), 38)],
 'shawn mendes': [((2022, 7), 64), ((2022, 5), 59), ((2022, 4), 39)],
 'zayn malik': [((2021, 4), 36), ((2021, 3), 34), ((2021, 2), 12)],
 'louis tomlinson': [((2022, 8), 27), ((2022, 11), 17), ((2022, 12), 15)],
 'taylor switf': [((2021, 9), 23), ((2021, 8), 18), ((2021, 6), 10)],
 '1989 taylor': [((2023, 8), 2), ((2023, 9), 2)],
 'niall horan': [((2022, 12), 15), ((2023, 2), 14), ((2023, 3), 12)],
 'tour barcelona': [((2023, 8), 8), ((2023, 7), 6), ((2023, 9), 1)],
 'tour 2023': [((2023, 8), 7), ((2023, 9), 2)],
 'tour madrid': [((2023, 7), 5), ((2023, 8), 5)],
 'stray kids': [((2023, 9), 1)],
 'justin bieber': [((2022, 6), 1)]}

In [111]:
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4689 entries, 0 to 6372
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Sale Date        4689 non-null   object 
 1   Item Name        4689 non-null   object 
 2   Buyer            4101 non-null   object 
 3   Quantity         4689 non-null   int64  
 4   Price            4689 non-null   float64
 5   Discount Amount  4689 non-null   float64
 6   Ship City        4689 non-null   object 
 7   Ship Country     4689 non-null   object 
 8   Palabras_clave   3168 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 366.3+ KB
